In [1]:
!pip3 install pbspark

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pbspark import from_protobuf
from driver_fatigue_pb2 import FatigueDetection
import sys

In [3]:
jarsPackages = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1"

In [4]:
spark = SparkSession.builder.master("local[*]") \
                    .appName('driver-fatigue-processor') \
                    .config("spark.jars.packages", jarsPackages) \
                    .getOrCreate()

22/09/13 19:24:52 WARN Utils: Your hostname, ubuntu-linux-20-04-desktop resolves to a loopback address: 127.0.1.1; using 10.211.55.8 instead (on interface eth0)
22/09/13 19:24:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/parallels/.ivy2/cache
The jars for the packages stored in: /home/parallels/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1394bdcc-3ac8-4a70-a535-ce79af64db88;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.1 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in c

22/09/13 19:24:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "103.146.34.72:9094") \
  .option("subscribe", "driver_fatigue_detection") \
  .option("includeHeaders", "true") \
  .load() \
  .selectExpr("value", "headers",'timestamp')

In [6]:
def message_proto(value):
    decoded = from_protobuf(value,FatigueDetection)
    print(decoded)
    return decoded

In [7]:
query = df \
        .withColumn('parsed',message_proto('value')) \
        .withColumn('bus_id', expr('headers')[2]['value'].cast('string')) \
        .select('bus_id','parsed.*') \
        .withColumn('timestamp',to_timestamp(col('timestamp')/1000)) \
        .withColumn('year', year('timestamp')) \
        .withColumn('month', month('timestamp')) \
        .withColumn('day', dayofmonth('timestamp')) \
        .withColumn('hour', hour('timestamp')) \
        .withColumn('minute', minute('timestamp')) \
        .withColumn('second',second('timestamp'))
        
query.printSchema()

Column<'decoder(value)'>
root
 |-- bus_id: string (nullable = true)
 |-- status: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- minute: integer (nullable = true)
 |-- second: integer (nullable = true)



In [8]:
# show = query \
#         .writeStream \
#         .queryName('RawDriverFatigue') \
#         .outputMode('append') \
#         .option('path','hdfs://localhost:9000/user/parallels/job/driver-fagitue/') \
#         .option('checkpointLocation','hdfs://localhost:9000/user/parallels/spark-checkpoint/driver-fatigue-checkpoint/') \
#         .partitionBy('year','month','day','hour','minute') \
#         .option("truncate", False) \
#         .start() \
#         .awaitTermination()

In [9]:
show = query \
        .writeStream \
        .queryName('RawDriverFatigue') \
        .outputMode('append') \
        .format('console') \
        .option("truncate", False) \
        .start() \
        .awaitTermination()

22/09/13 19:24:56 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-c2c14b55-49c4-493c-90ee-596309c11976. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/09/13 19:24:57 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
-------------------------------------------
Batch: 0
-------------------------------------------
+------+------+---------+----+-----+---+----+------+------+
|bus_id|status|timestamp|year|month|day|hour|minute|second|
+------+------+---------+----+-----+---+----+------+------+
+------+------+---------+----+-----+---+----+------+------+



22/09/13 19:27:30 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
-------------------------------------------
Batch: 1
-------------------------------------------
+------+------+-----------------------+----+-----+---+----+------+------+
|bus_id|status|timestamp              |year|month|day|hour|minute|second|
+------+------+-----------------------+----+-----+---+----+------+------+
|bus_42|null  |2022-09-13 19:27:29.138|2022|9    |13 |19  |27    |29    |
+------+------+-----------------------+----+-----+---+----+------+------+



22/09/13 19:27:31 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
22/09/13 19:27:31 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
-------------------------------------------
Batch: 2
-------------------------------------------
+------+------+-----------------------+----+-----+---+----+------+------+
|bus_id|status|timestamp              |year|month|day|hour|minute|second|
+------+------+-----------------------+----+-----+---+----+------+------+
|bus_42|2     |2022-09-13 19:27:30.144|2022|9    |13 |19  |27    |30    |
|bus_42|null  |2022-09-13 19:27:31.154|2022|9    |13 |19  |27    |31    |
+------+------+-----------------------+----+-----+---+----+------+------+

22/09/13 19:27:32 WARN KafkaDataConsumer: KafkaDataConsumer is not running in Uninterrupti


[Stage 3:>                                                          (0 + 1) / 1]



-------------------------------------------
Batch: 3
-------------------------------------------
+------+------+----------------------+----+-----+---+----+------+------+
|bus_id|status|timestamp             |year|month|day|hour|minute|second|
+------+------+----------------------+----+-----+---+----+------+------+
|bus_42|1     |2022-09-13 19:27:32.16|2022|9    |13 |19  |27    |32    |
+------+------+----------------------+----+-----+---+----+------+------+

22/09/13 19:27:33 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894


...


[Stage 19:>                                                         (0 + 1) / 1]



-------------------------------------------
Batch: 19
-------------------------------------------
+------+------+-----------------------+----+-----+---+----+------+------+
|bus_id|status|timestamp              |year|month|day|hour|minute|second|
+------+------+-----------------------+----+-----+---+----+------+------+
|bus_42|null  |2022-09-13 19:27:48.342|2022|9    |13 |19  |27    |48    |
+------+------+-----------------------+----+-----+---+----+------+------+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/parallels/anaconda3/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/parallels/anaconda3/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/parallels/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


22/09/13 19:27:49 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894


KeyboardInterrupt: 